# Examples of usage:

In [ ]:
import os,sys
sys.path.append(os.path.abspath(".."))
from tool_box import tool_box
import pandas as pd
from pandas import DataFrame

###  DICOM to nrrd conversion

In [ ]:
##Convert DICOM dataset to the volume (nrrd) format
# export_path = ... export path where the converted nrrds will be placed, region_of_interest = ... if you know exact name of
# the ROI you want to extract then write it with the ! character infront, eg. region_of_interest = !gtv1 , 
# if you want to extract all the gtv's in the rtstructure eg. gtv1, gtv2, gtv_whatever then just specify the stem word 
# eg. region_of_interest = gtv, default value is region_of_interest ='all', where all ROI's in rtstruct will be extracted.
        

In [ ]:
# Set up the parameters for the 
parameters = {'data_path': r'../data/dcms/', #path_to_your_data
              'data_type': 'dcm',
              'multi_rts_per_pat': False}   #When False, it will look only for 1 rtstruct in the patient folder, this will
                                            #speed up the process, if you have more then 1 rtstruct per patient, set it to True
                  
export_path =r'../data/'    

In [ ]:
#Inintialize the dataset 
Data_MRI = tool_box(**parameters) 

In [ ]:
#Run the conversion
Data_MRI.convert_to_nrrd(export_path,'gtv')

### Quick check of the ROI's in the nrrd dataset

In [ ]:
#If you want to check your converted images&ROIs or just any volumetric dataset (nrrd/mha), you can use get_jpegs function of
#the toolbox, it will generate jpeg images for each slice with the image and overlap of the contour.

In [ ]:
Data_MRI_nrrd = tool_box(data_path = r'../data/converted_nrrds/',
                    data_type='nrrd')

In [ ]:
Data_MRI_nrrd.get_jpegs(r'../data/')

In [ ]:
#Lets just check one of the patient's right in jupyter

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np
from PIL import Image

def browse_images(images,names):
    n = len(images)
    def view_image(i):
        plt.figure(figsize=(20,10))
        plt.imshow(images[i])#, cmap=plt.cm.gray_r, interpolation='nearest')
        plt.title('Slice: %s' % names[i])
        plt.axis('off')
        plt.show()
    interact(view_image, i=(0,n-1))
    
for pat,_ in Data_MRI_nrrd:
    _,file_struct = [*os.walk(os.path.join('../data/images_quick_check/',pat))]
    root,images = file_struct[0],file_struct[2]
    imgs =[np.array(Image.open(os.path.join(root,img))) for img in images]
    print(pat)
    browse_images(imgs,images)
    break


    


### Extract PyRadiomics features

In [ ]:
Data_MRI_nrrd = tool_box(data_path = r'../data/converted_nrrds/',
                    data_type='nrrd')

In [ ]:
#Use your own pyradiomics parameters file, more you can find here: https://github.com/Radiomics/pyradiomics 
parameters = r"../example_ct_parameters.yaml"
features = Data_MRI_nrrd.extract_features(parameters,loggenabled=True)

In [ ]:
features.head(5)

In [ ]:
#Write features to excel file
writer = pd.ExcelWriter('../data/features/extracted_features.xlsx')
features.to_excel(writer,'Sheet1')
writer.save()